In [34]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
from datetime import datetime
from pathlib import Path
import pyproj

In [36]:
class ActivityHandler:
    def __init__(self):
        pass

    def build_path(self, points):
        # Build a LineString from a list of coordinate pairs
        line = LineString(points)
        print(f"Number of points used to create the geometry: {len(points)}")
        return line

    def calculate_stats(self, line, start_time, end_time):
        # Calculate trip duration in hours
        duration = (end_time - start_time).total_seconds() / 3600.0

        # Project LineString to calculate distance in meters, then convert to miles
        line_geoseries = gpd.GeoSeries([line], crs='EPSG:4326')
        line_projected = line_geoseries.to_crs(epsg=3857)  # Project to a metric system (Web Mercator)
        distance_meters = line_projected.length.iloc[0]
        distance_miles = distance_meters / 1609.34

        # Calculate average speed in miles per hour
        avg_speed = distance_miles / duration

        # Calculate average pace in minutes per mile
        avg_pace = (duration * 60) / distance_miles

        # Create a dictionary with the calculated stats
        stats = {
            "distance_miles": distance_miles,
            "avg_speed_mph": avg_speed,
            "avg_pace_min_per_mile": avg_pace
        }

        # Print the statistics
        print(f"Distance: {distance_miles:.2f} miles")
        print(f"Average Speed: {avg_speed:.2f} mph")
        print(f"Average Pace: {avg_pace:.2f} min/mi")

        return stats

    def get_centroid(self, geometry):
        # Calculate the centroid of the given geometry
        centroid = geometry.centroid
        print(f"Centroid coordinates: {centroid.x:.6f}, {centroid.y:.6f}")
        return centroid

    def export_activity(self, path, stats):
        # Create a GeoDataFrame from the LineString and export it as a shapefile
        gdf = gpd.GeoDataFrame([stats], geometry=[stats['path']], crs='EPSG:4326')
        gdf.to_file(path, driver='ESRI Shapefile')
        print(f"Activity exported to {path}")

In [38]:
if __name__ == '__main__':
    # Set up paths
    wkDir = Path(r"C:\Users\nstra\iCloudDrive\Desktop\School\GIS6103 GIS Programming and Customization\Assignments\Assignment-11")
    csv_path = wkDir / 'gps_track.csv'

    # Create lists to store coordinate pairs and timestamps
    coordinate_list = []
    timestamps = []

    # Open CSV file to iterate through rows to extract longitude, latitude, and time
    with open(csv_path, 'r') as file:
        next(file)  # Skip the header
        for line in file:
            parts = line.strip().split(',')
            # Update column indices based on the CSV structure
            ele, time, lat, lon = float(parts[2]), parts[3], float(parts[4]), float(parts[5])
            coordinate_list.append((lon, lat))
            # Adjust format to match the CSV's timestamp format
            timestamps.append(datetime.strptime(time, "%Y/%m/%d %H:%M:%S.%f"))



    # Create an instance of ActivityHandler
    activity = ActivityHandler()

    # Build a LineString from input points
    path = activity.build_path(coordinate_list)

    # Calculate trip statistics
    start_time = timestamps[0]
    end_time = timestamps[-1]
    stats = activity.calculate_stats(path, start_time, end_time)
    stats['path'] = path  # Add path to stats dictionary

    # Get the centroid of the trip
    centroid = activity.get_centroid(path)

    def export_activity(self, path, stats):
        # Extract the path (geometry) and remove it from the stats dictionary
        line_geometry = stats.pop('path')
    
        # Create a GeoDataFrame from the attributes (stats) and the geometry
        gdf = gpd.GeoDataFrame([stats], geometry=[line_geometry], crs='EPSG:4326')
    
        # Export the GeoDataFrame to a Shapefile
        gdf.to_file(path, driver='ESRI Shapefile')
        print(f"Activity exported to {path}")
        
        print(gdf)


Number of points used to create the geometry: 982
Distance: 9.92 miles
Average Speed: 6.12 mph
Average Pace: 9.80 min/mi
Centroid coordinates: -80.355266, 25.772596
